# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686605


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:31,  3.25s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:02,  2.33s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:43,  1.67s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:30,  1.21s/it]

...

Rendering models:  77%|███████▋  | 23/30 [00:08<00:01,  6.76it/s]

Rendering models:  83%|████████▎ | 25/30 [00:08<00:00,  7.65it/s]

Rendering models:  90%|█████████ | 27/30 [00:08<00:00,  9.17it/s]

Rendering models:  97%|█████████▋| 29/30 [00:08<00:00, 10.13it/s]

not-logged-in-68d3d82149e88f995533    0.584492
zac8151                               0.000941
Petrusperes                           0.001913
CThomas                               0.014962
shirogane                             0.001045
tanya_p                               0.000627
Andy_Carleen                          0.000810
KG.1612                               0.000704
Dynamatt                              0.001221
bryndismr                             0.005029
not-logged-in-b77ab2eec9cd437a5d6d    1.495800
pangeli5                              0.002006
acapirala                             0.014170
dinosora                              0.001049
rhondacard                            0.005704
Lavadude                              0.013230
helios1235                            0.000553
ShirleyL                              0.000609
nicholaschua                          0.001869
not-logged-in-15498ebaa693dd5a198d    0.000643
not-logged-in-0a5c46abe4a4b6d51e31    0.122807
Tessaje04    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())